# Mask-RCNN Model
In this notebook I examine the performance of Maks-RCNN

In [ ]:
import os
import sys
sys.path.insert(0, "..")
import gc
gc.enable()

In [ ]:
sys.path.insert(0, os.getcwd())
from models.mask_rcnn import MRCNN
from utils.utilities import *
import tensorflow as tf
import gc
gc.enable()
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)

from utils.DataLoader import  DataLoader
from models.mask_rcnn import DetectorDataset

### Initializing Mask-RCNN's data Loader based on DataLoader

In [ ]:
dl = DataLoader()
train_df, valid_df = dl.train_split(0.05, filterNan=True)

dd_train = DetectorDataset(train_df, shape=(256, 256))
dd_train.prepare()
dd_valid = DetectorDataset(valid_df, shape=(256, 256))
dd_valid.prepare()

print(train_df.shape)
print(valid_df.shape)

### An example of Mask-RCNN data augmentation

In [ ]:
import random
import matplotlib.pyplot as plt

class_ids = [0]
while class_ids[0] == 0:  ## look for a mask
    image_id = random.choice(dd_valid.image_ids)
    image_fp = dd_valid.image_reference(image_id)
    image = dd_valid.load_image(image_id)
    mask, class_ids = dd_valid.load_mask(image_id)

print(image.shape)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 2, 2)
masked = np.zeros(image.shape[:2])
for i in range(mask.shape[2]):
    masked += mask[:, :, i] ## * image[:, :, 0]
plt.imshow(masked, cmap='gray')
plt.axis('off')

print(image_fp)
print(class_ids)

### Loading Mask-RCNN and Training

In [ ]:
mrcnn = MRCNN(model_folder="../models/serialized/")
h = mrcnn.train(dd_train, dd_valid, epochs=15, layers='all')
mrcnn.show_loss(h)

### Loading testing set and examining Mask-RCNN performance

In [ ]:
#mrcnn = MRCNN('/home/gmandi/Documents/DSIT/Deep-Learning/AIRBUS-Ship_Detection/models/serialized/mask-rcnn20200917T2003/mask_rcnn_mask-rcnn_0010.h5', model_folder="../models/serialized/")
dd_test = DetectorDataset(dl.test, shape=(256, 256))
dd_test.prepare()
len(dd_test.image_ids)

mrcnn.examine_performance(dd_test)